In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist

In [ ]:
# plot speed per condition will be interesting to see and assess 
